In [ ]:
!pip install transformers faiss-cpu torch
!pip install pdfplumber
import pandas as pd
import re
import io
import pdfplumber
import requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 92.2 MB/s eta 0:00:00


In [ ]:
ATTACHMENT_TYPE_ERR_MSG = (
    "All attachments must be either DctmObjRef or Attachment type, got {}: {}"
)
ATTACH_TYPE_EXPECTED = "Attachment expected to be of type `Attachment`, got {}"
UNEXPECTED_ATTR_TO_PARSE = (
    "Attribute to parse from attachments expected to be in "
    "['body', 'filename'], got '{}'"
)
DCTM_OBJ_REF_EXPECTED = "Expected DctmObjRef, got {}: {}"
DOXC2TXT_EXCEPTION = "Cannot process file, raised '{}' error"
LIST_OR_STR_ATTACH_EXPECTED = "Got type {} for attachment, only list or str accepted"
PAGE_SEP = "\n" + "=" * 31 + " NEW PAGE " + "=" * 31 + "\n"
MISSING_SPACES_PATTERNS = [
    "IndicativeTermsheet\n",
    "PRIVATEPLACEMENT\n",
    "PublicOfferingonlyin:",
]


def check_txt_missing_spaces(all_pages_txt: str, threshold: float = 0.06) -> bool:
    """Check if the parsed PDF has missing spaces (as for all Leonteq termsheets).

    Notes
    -----
    The alignment used to format the Leonteq termsheets are not properly recognized by
    our PDF converter. As an undesirable result, most spaces are being removed during
    the conversion step leading to erroneous extractions.
    """

    nb_spaces = all_pages_txt.count(" ")
    nb_chars = len(all_pages_txt)
    ratio = nb_spaces / nb_chars

    return ratio < threshold and any(
        p in all_pages_txt for p in MISSING_SPACES_PATTERNS
    )


def pdf_text_from_bytes(
    pdf_bytes_string: bytes,
    max_pages: int = 999,
    pages_sep: str = PAGE_SEP,
) -> str:
    """Convert the PDF byte representation to text."""
    try:
        # Pdfplumber returns empty string for UTF-8 encoded strings
        # (without any exception raised), only Latin-1 works
        # On the other hand, FastAPI requires UTF-8 strings in payloads,
        # so we assume UTF-8 string arrives here
        pdf_bytes_string = pdf_bytes_string.decode("UTF-8").encode("Latin1")
    except UnicodeDecodeError:
        # If the above command fails, we will assume the byte string
        # is already Latin1 encoded
        pass

    all_pages_txt = ""
    pages_list = []
    with pdfplumber.open(io.BytesIO(pdf_bytes_string)) as pdf:
        for page_idx in range(
            0, min(len(pdf.pages), max_pages)
        ):  # pylint: disable=invalid-name
            pages_list.append(pdf.pages[page_idx].extract_text() + pages_sep)
            all_pages_txt += pdf.pages[page_idx].extract_text() + pages_sep
    return pages_list

In [ ]:
with open(
    "/content/llama2.pdf",
    "rb",
) as fobj:
    pdf_bytes_utf8 = fobj.read().decode("Latin1").encode("UTF-8")
    pdf_text = pdf_text_from_bytes(pdf_bytes_utf8)


def clean_text(text):

    text = text.lower()
    text = text.replace("\n", " ")
    text = text.replace("  ", " ")
    return text


cleaned_txt = []
for page in pdf_text:
    cleaned_txt.append(clean_text(page))

In [ ]:
import faiss
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import normalize
import numpy as np

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")


def embed_text(text, tokenizer, model):
    # Tokenize and get model embeddings
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # mean pooling
    return embeddings.numpy()


# Assume `pdf_pages` is a list of strings where each string is the text of one page of your PDF
pdf_pages = cleaned_txt.copy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [ ]:
# Step 1: Embed all pages and store them in a list
page_embeddings = [embed_text(page, tokenizer, model) for page in pdf_pages]
page_embeddings = np.vstack(page_embeddings)  # Stack embeddings into a single array

In [ ]:
# Step 2: Normalize embeddings for cosine similarity
normalized_embeddings = normalize(page_embeddings)

# Step 3: Create a FAISS index and add embeddings
dimension = normalized_embeddings.shape[1]
index = faiss.IndexFlatIP(
    dimension
)  # IP for inner product, since embeddings are normalized
index.add(normalized_embeddings)


In [ ]:

# Function to perform similarity search
def search_similar_pages(query, index, tokenizer, model, k=3):
    query_embedding = embed_text(query, tokenizer, model)
    query_embedding = normalize(query_embedding)  # Normalize query embedding
    _, top_k_indices = index.search(query_embedding, k)
    return top_k_indices[0]


# Example query
query = "who are the writers of this paper"
top_k_pages = search_similar_pages(query, index, tokenizer, model, k=3)

# Output relevant pages
for i in top_k_pages:
    print(f"Relevant page {i+1}: {pdf_pages[i]}")

Relevant page 45: terryyuezhuo,yujinhuang,chunyangchen,andzhenchangxing. exploringaiethicsofchatgpt: a diagnosticanalysis. arxivpreprintarxiv:2301.12867,2023. 45 =============================== new page =============================== 
Relevant page 39: adamroberts,dennyzhou,quocv.le,andjasonwei. scalinginstruction-finetunedlanguagemodels. arxivpreprintarxiv:2210.11416,2022. christopherclark,kentonlee,ming-weichang,tomkwiatkowski,michaelcollins,andkristinatoutanova. boolq: exploringthesurprisingdifficultyofnaturalyes/noquestions. arxivpreprintarxiv:1905.10044, 2019. elizabethclark,talaugust,sofiaserrano,nikitahaduong,suchingururangan,andnoaha.smith.allthat’s ‘human’isnotgold: evaluatinghumanevaluationofgeneratedtext.inproceedingsofthe59thannualmeeting oftheassociationforcomputationallinguisticsandthe11thinternationaljointconferenceonnaturallanguage processing(volume1: longpapers),pages7282–7296,online,august2021.associationforcomputational linguistics. doi: 10.18653/v1/2021.acl-long.56

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

messages = [{"role": "user", "content": "What is the capital of France."}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>


In [ ]:
messages = [{"role": "user", "content": "what can you do for me"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_new_tokens=50, temperature=0.01, top_p=0.9, do_sample=True)
outputs = model.generate(inputs, max_new_tokens=10000)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
what can you do for me<|im_end|>
<|im_start|>assistant
I can assist you in a variety of ways. Here are some of the things I can do:

1. **Answer questions**: I can provide information on a wide range of topics. Just ask me a question and I'll do my best to provide a clear and concise answer.

2. **Explain concepts**: If you're having trouble understanding a concept or idea, I can help explain it to you.

3. **Provide information**: I can provide information on a wide range of topics, from science and history to pop culture and more.

4. **Help with homework**: If you're struggling with a particular assignment or concept, I can help explain it to you.

5. **Discuss topics**: I can engage in a conversation about a wide range of topics. Whether you're interested in learning more about a particular subject or just want to chat, I'm here to help.

6. **Provide resources**: If y

In [ ]:
def llm(st):
  messages = [{"role": "user", "content": "What is the capital of France."}]
  input_text=tokenizer.apply_chat_template(messages, tokenize=False)
  inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
  outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
  print(tokenizer.decode(outputs[0]))

In [ ]:
tokenizer.decode(outputs[0])

"<|im_start|>system\nYou are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>\n<|im_start|>user\nwhat can you do for me<|im_end|>\n<|im_start|>assistant\nI can assist you in a variety of ways. Here are some of the things I can do:\n\n1. **Answer questions**: I can provide information on a wide range of topics. Just ask me a question and I'll do my best to provide a clear and concise answer.\n\n2. **Explain concepts**: If you're having trouble understanding a concept or idea, I can help explain it to you.\n\n3. **Provide information**: I can provide information on a wide range of topics, from science and history to pop culture and more.\n\n4. **Help with homework**: If you're struggling with a particular assignment or concept, I can help explain it to you.\n\n5. **Discuss topics**: I can engage in a conversation about a wide range of topics. Whether you're interested in learning more about a particular subject or just want to chat, I'm here to help.\n\n6. **Provide

In [ ]:
zit = """
extract all the names of the scientits mentioned in the article below :

"""
tot = zit + pdf_pages[0]

In [ ]:
tot

'\nextract all the names of the scientits mentioned in the article below : \n\nllama 2: open foundation and fine-tuned chat models hugotouvron∗ louismartin† kevinstone† peteralbert amjadalmahairi yasminebabaei nikolaybashlykov soumyabatra prajjwalbhargava shrutibhosale danbikel lukasblecher cristiancantonferrer moyachen guillemcucurull davidesiobu judefernandes jeremyfu wenyinfu brianfuller cynthiagao vedanujgoswami namangoyal anthonyhartshorn sagharhosseini ruihou hakaninan marcinkardas viktorkerkez madiankhabsa isabelkloumann artemkorenev punitsinghkoura marie-annelachaux thibautlavril jenyalee dianaliskovich yinghailu yuningmao xaviermartinet todormihaylov pushkarmishra igormolybog yixinnie andrewpoulton jeremyreizenstein rashirungta kalyansaladi alanschelten ruansilva ericmichaelsmith ranjansubramanian xiaoqingellentan binhtang rosstaylor adinawilliams jianxiangkuan puxinxu zhengyan iliyanzarov yuchenzhang angelafan melaniekambadur sharannarang aurelienrodriguez robertstojnic serge

In [ ]:
messages = [{"role": "user", "content": tot}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_new_tokens=50, temperature=0.01, top_p=0.9, do_sample=True)
outputs = model.generate(inputs, max_new_tokens=10000)
print(tokenizer.decode(outputs[0]))

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (8192). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user

extract all the names of the scientits mentioned in the article below : 

llama 2: open foundation and fine-tuned chat models hugotouvron∗ louismartin† kevinstone† peteralbert amjadalmahairi yasminebabaei nikolaybashlykov soumyabatra prajjwalbhargava shrutibhosale danbikel lukasblecher cristiancantonferrer moyachen guillemcucurull davidesiobu judefernandes jeremyfu wenyinfu brianfuller cynthiagao vedanujgoswami namangoyal anthonyhartshorn sagharhosseini ruihou hakaninan marcinkardas viktorkerkez madiankhabsa isabelkloumann artemkorenev punitsinghkoura marie-annelachaux thibautlavril jenyalee dianaliskovich yinghailu yuningmao xaviermartinet todormihaylov pushkarmishra igormolybog yixinnie andrewpoulton jeremyreizenstein rashirungta kalyansaladi alanschelten ruansilva ericmichaelsmith ranjansubramanian xiaoqingellentan binhtang rosstaylor adinawilliams jianxiangkuan puxinx